In [2]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [3]:
# Load YAML file
with open('custom_data.yaml',mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)

labels = data_yaml['names']
print(labels)



['battery', 'biological', 'cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [4]:
# Load YOLO Model
yolo = cv2.dnn.readNetFromONNX('./Model7/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [5]:
# Load the image
img = cv2.imread('./photo.jpg')
image = img.copy()
row, col, d = image.shape

# get the YOLO prediction from the image
# step-1 convert image into square image (array)
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype = np.uint8)
input_image[0:row,0:col] = image

# step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB = True, crop = False)
yolo.setInput(blob)
predits = yolo.forward()  #detection or prediction from YOLO Model

# cv2.imshow('photo',input_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [6]:
print(predits.shape)

(1, 25200, 13)


In [7]:
# Non Maximum Supression
# sept-1: filter detection based on confidenc (0.4) and probability score (0.25)
detections = predits[0]
boxes = []
confidences = []
classes = []

# width and height of the image (photo)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]  #confidnece of detection on object
    if confidence > 0.2:
        class_score = row[5:].max()  #maximum probility from 13 objects
        class_id = row[5:].argmax() #get the index position at which max probability occur

        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            # construct bounding box from four values
            # left, top, width and height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)

            box = np.array([left, top, width, height])

            # oppend values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()


In [8]:
confidences

[0.51690817,
 0.26757056,
 0.31109035,
 0.29531336,
 0.23173356,
 0.5351003,
 0.28701606,
 0.25234282,
 0.3242468,
 0.27934894,
 0.22900656]

In [9]:
boxes

[array([   1,   36, 1104,  912]),
 array([348,  66, 653, 869]),
 array([  -7,   61, 1113,  878]),
 array([367,  90, 646, 854]),
 array([375, 100, 644, 846]),
 array([  15,   36, 1077,  913]),
 array([  47,   42, 1057,  905]),
 array([343,  64, 660, 872]),
 array([  -4,   66, 1106,  868]),
 array([367,  87, 642, 859]),
 array([377,  98, 638, 845])]

In [21]:
classes

[5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 5,
 6,
 5,
 5,
 5,
 5,
 6,
 5,
 5,
 6]

In [22]:
index

array([65, 43,  2, 41, 64, 72], dtype=int32)

In [23]:
len(index)

6

In [10]:
# Draw the Bounding Box
for ind in index:
    # extract bounding box
    x,y,w,h = boxes_np[ind]
    boxb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]

    text = f'{class_name}: {boxb_conf}%'
    # print(text)

    # for rectangel 
    cv2.rectangle(image,(x,y),(x+w,y+h), (0,255,0),2)
    cv2.rectangle(image,(x,y-30), (x+w,y), (255,255,255),-1)

    # for Text
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)


In [11]:
cv2.imshow('original',img)
cv2.imshow('data_predictions_yolo', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

2024-07-12 23:27:08.450 Python[4956:2669891] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 